autograde

深度学习的算法本质上是通过反向传播求导数，而PyTorch的autograd模块则实现了此功能。在Tensor上的所有操作，autograd都能为它们自动提供微分，避免了手动计算导数的复杂过程。  
https://github.com/chenyuntc/pytorch-book/blob/master/chapter03-tensor_and_autograd/Autograd.ipynb

# 使能autograde

要想使得Tensor使用autograd功能，只需要设置tensor.requries_grad=True.从这个tensor之后的所有tensor都具有autograde功能.
可以在创建tensor的时候加上 requires_grad=True,也可以 使用tensor.requires_grad=True 来开启autograde功能

In [2]:
import torch as tf

In [7]:
x = tf.ones(2,3,requires_grad=True)
print(x)

tensor([[1., 1., 1.],
        [1., 1., 1.]], requires_grad=True)


In [8]:
x = tf.ones(2,3)
print(x)
x.requires_grad = True
print(x)

tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([[1., 1., 1.],
        [1., 1., 1.]], requires_grad=True)


# 使用autograde

In [43]:
import torch as tf

在设置requires_grad=True之后,当使用这个tensor进行运算时,pytorch就会自动生成对应的梯度函数

In [44]:
x = tf.ones(3,3,requires_grad=True)
print(x.grad_fn) # 因为没有计算,所以grad_fn是None

None


In [45]:
y = x ** 2
print(y.grad_fn) # 指数计算grad_fn

In [46]:
z = (y + 1)* y
print(z.grad_fn,z)

<MulBackward0 object at 0x7fb281656dd8> tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]], grad_fn=<MulBackward0>)


## 对单个元素进行反向传播

In [47]:
z.backward() # 反向传播必须从0维tensor开始,所以这样会报错

RuntimeError: grad can be implicitly created only for scalar outputs

In [48]:
z[0][0].backward() # 可以对单个元素执行反向传播
x.grad

tensor([[6., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [50]:
z[0][0].backward() # 每执行一次计算,就只能做依次反向传播,重复做会报错

RuntimeError: Trying to backward through the graph a second time, but the saved intermediate results have already been freed. Specify retain_graph=True when calling backward the first time.

## 如何对所有的元素进行反向传播?

为了对所有的元素进行反向传播,那就需在最后一步计算的时候让所有的元素都参与计算,且不改变原来的梯度值.sum()正好可以满足这个条件.
所以在计算的最后一步对最后的tensor进行一次求和操作,再对这个和执行 backward()

In [52]:
import torch as t
x = t.ones(3,2,requires_grad=True)
y = x**2
z = (y + 1)* y
s = z.sum()
print(s)
s.backward()
x.grad

tensor(12., grad_fn=<SumBackward0>)


tensor([[6., 6.],
        [6., 6.],
        [6., 6.]])

## grad 的自动累加

grad会自动累加的,所以执行两次计算,会把前一次的grade累加进来. 这不是反向传播需要的结果

In [56]:
import torch as t
x = t.ones(3,2,requires_grad=True)
y = x**2
z = (y + 1)* y
s = z.sum()
print(s)
s.backward()
print(x.grad)
y = x**2
z = (y + 1)* y
s = z.sum()
print(s)
s.backward()
print(x.grad) # x.grad 会变成12

tensor(12., grad_fn=<SumBackward0>)
tensor([[6., 6.],
        [6., 6.],
        [6., 6.]])
tensor(12., grad_fn=<SumBackward0>)
tensor([[12., 12.],
        [12., 12.],
        [12., 12.]])


In [57]:
# 在执行backward之前,要清除grad
import torch as t
x = t.ones(3,2,requires_grad=True)
y = x**2
z = (y + 1)* y
s = z.sum()
print(s)
s.backward()
print(x.grad)
y = x**2
z = (y + 1)* y
s = z.sum()
print(s)
x.grad.data.zero_()
s.backward()
print(x.grad) # x.grad 依然是6

tensor(12., grad_fn=<SumBackward0>)
tensor([[6., 6.],
        [6., 6.],
        [6., 6.]])
tensor(12., grad_fn=<SumBackward0>)
tensor([[6., 6.],
        [6., 6.],
        [6., 6.]])


## 中间节点的grad

In [ ]:
通常情况下只能得到叶子节点的grad,无法得到中间节点的grad.
但是在训练的时候需要能更新所有节点的权重,该如何得到中间节点的grad呢?

In [59]:
import torch as t
x = t.ones(3,2,requires_grad=True)
y = x**2
z = (y + 1)* y
s = z.sum()
print(s)
s.backward()
print(y.grad)


tensor(12., grad_fn=<SumBackward0>)
None


/home/gang/anaconda3/envs/YOLO/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the gradient for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations.
  


# Autograde